<a href="https://colab.research.google.com/github/435wldms/essa/blob/main/%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


배치 경사 하강법, 확률적 경사 하강법, 미니 배치 경사 하강법을 주로 사용한다.
>
정규방정식은 계산 복잡도가 특성의 개수에 따라 매우 빠르게 증가하기에 사용할 수 없다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

train data에 과적합되고 있음을 의미한다. 이럴 때는 검증 에러가 최솟값에 도달했을 때 학습을 조기 종료하면 된다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

모델이 훈련 데이터에 과소 적합되었을 가능성이 크다. 이때는 높은 편향이 문제이다. 그러므로 alpha를 감소시켜야 분산이 증가하고 편향이 감소한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

약간의 규제가 있는 것이 좋기 때문에 릿지 회귀가 나음.
>
라쏘 회귀는 릿지와는 달리 덜 중요한 특성의 가중치를 제거해 희소한 모델을 만들어 줌. -> 몇 개의 특성만 유용할 것이라 의심될 때 사용하면 좋음.
>
특성 수가 훈련 샘플 수보다 많거나 특성 몇 개가 강하게 연관되어 있을 때는 라쏘가 문제를 일으킬 수 있으므로 엘라스틱넷을 선호함.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [ ]:
import numpy as np
def softmax(logits):
  exps = np.exp(logits)
  exp_sums = np.sum(exps, axis=1, keepdims=True)
  return exps / exp_sums

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid=train_test_split(X,y, test_size=0.2)

In [ ]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train)) # 3개의 붓꽃 클래스

eta = 0.1
m = len(X_train)
iteration= 5001
epsilon = 1e-7
alpha = 0.1
best_loss = np.infty

Theta = np.random.randn(n_inputs,n_outputs)

for i in range(iteration):
    logits = X_train.dot(Theta) # x벡터 특성 수 * y class 수
    Y_proba = softmax(logits) # 예측값
    entropy_loss = -np.mean(np.sum(y_train * np.log(Y_proba + epsilon), axis=1)) # 크로스 엔트로피 비용 함수
    l2_loss = 1/2 * np.sum(np.square(Theta[1:])) # l2 규제
    loss = entropy_loss + alpha * l2_loss # l2 규제가 추가된 손실
    error = Y_proba - y_train # 클래스 k에 속할 확률과 실제 간 차이
    l2_loss_gradient = np.r_[(np.zeros([1,n_outputs]),alpha * Theta[1:])]
    gradients = 1/m * X_train.T.dot(error) + l2_loss_gradient # 일반 그레디언트 + l2 규제 그레디언트
    Theta = Theta - eta * gradients # theta - 학습률*그레디언트 벡터

    logits = X_valid.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(y_valid * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 500 == 0:
        print(i,loss)
    if loss < best_loss: # 조기 종료 구현
        best_loss = loss
    else:
        print(i-1, best_loss)
        print(i,loss,"Early Stopping!")
        break